In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
query_gender = """

SELECT gender_segment
FROM user
"""

In [4]:
df_gender = pd.read_sql(query_gender, engine) 
display(df_gender.value_counts())

gender_segment
1.0               272442
0.0               228750
dtype: int64

In [6]:
query_all = """
WITH 

age_seg AS (SELECT age_gr_id,
                   title AS age_segment
           FROM age_segment),

traffic_seg AS (SELECT tr_gr_id,
                       title AS traffic_segment
                FROM traffic_segment),

lifetime_seg AS (SELECT lt_gr_id,
                   title AS lifetime_segment
           FROM lifetime_segment)

SELECT u.user_id AS user_id,
       u.lt_day AS lt_day,
       CASE
           WHEN u.lt_day <= 365 THEN 'new'
           ELSE 'old'
       END is_new,
       u.age AS age,
       u.gender_segment as raw_gender,
       CASE
           WHEN u.gender_segment == 1 THEN 'женщина'
           WHEN u.gender_segment == 0 THEN 'мужчина'
           ELSE 'unknown'
       END gender_segment,
       u.os_name AS os_name,
       u.cpe_type_name AS cpe_type_name,
       l.country AS country,
       l.city AS city,
       a.age_segment AS age_segment,
       t.traffic_segment AS traffic_segment,
       l.lifetime_segment AS lifetime_segment,
       u.nps_score AS nps_score,
       CASE
           WHEN nps_score >= 9 THEN 'сторонник'
           WHEN nps_score <=8 AND nps_score >=7 THEN 'нейтрал'
           WHEN nps_score <= 6 THEN 'критик'
       END nps_group       
FROM user AS u LEFT JOIN location AS l ON u.location_id=l.location_id
LEFT JOIN age_seg AS a ON u.age_gr_id=a.age_gr_id
LEFT JOIN traffic_seg AS t ON u.tr_gr_id=t.tr_gr_id
LEFT JOIN lifetime_seg AS l ON u.lt_gr_id=l.lt_gr_id
              
"""

In [7]:
df_all = pd.read_sql(query_all, engine) 
display(df_all.head(10))

,user_id,lt_day,is_new,age,raw_gender,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,1.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,сторонник
1,A001WF,2344,old,53.0,0.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,сторонник
2,A003Q7,467,old,57.0,0.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,сторонник
3,A004TB,4190,old,44.0,1.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,сторонник
4,A004XT,1163,old,24.0,0.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,сторонник
5,A005O0,5501,old,42.0,1.0,женщина,ANDROID,SMARTPHONE,Россия,Омск,04 35-44,05 5-10,08 36+,6,критик
6,A0061R,1236,old,45.0,0.0,мужчина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,06 10-15,08 36+,10,сторонник
7,A009KS,313,new,35.0,0.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,04 35-44,13 45-50,05 7-12,10,сторонник
8,A00AES,3238,old,36.0,1.0,женщина,ANDROID,SMARTPHONE,Россия,СанктПетербург,04 35-44,04 1-5,08 36+,10,сторонник
9,A00F70,4479,old,54.0,1.0,женщина,ANDROID,SMARTPHONE,Россия,Волгоград,05 45-54,07 15-20,08 36+,9,сторонник


In [8]:
df_all.to_csv('telecomm_csi_tableau.csv', index=False)

### Укажите ссылку на дашборд на сайте Tableau Public:

https://public.tableau.com/app/profile/sofia3695/viz/NPS_16764753828630/Dashboard1?publish=yes

### Укажите ссылку на pdf-файл с презентацией:

https://drive.google.com/file/d/1CosS61c1bCfpA2Zn5RnrVjPx7HQZR6GN/view?usp=sharing